In [1]:
%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:60% !important; }</style>"))

In [3]:
from visualize_pyphi import *
from visualize_pyphi import utils
from visualize_pyphi import compute
import pickle as pkl
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

# Make network

In [4]:
# Chapter 3 network

net_name = "ch3"
n=6

# system parameters
strong = 1
medium = 0.2
weak = 0.04
none = 0

# input output
ff = 0.05
fb = 0.005

# naka rushdon params
th = 1 / 4
exp = 5

weights = np.array(
    [
        [strong, none, weak, medium, 0, 0],
        [medium, weak, none, strong, fb, 0],
        [weak, strong, medium, none, 0, 0],
        [none, medium, strong, weak, 0, ff],
        [0, ff, 0, 0, 0, 0],
        [0, 0, 0, fb, 0, 0],
    ]
)
weights[:4,:4] /= weights[:4,:4].sum(1)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = ["s"] * n

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=False,
)

  0%|          | 0/64 [00:00<?, ?it/s]

# Plot weights and tpms

In [5]:
wdf = pd.DataFrame(np.round(weights,3),index=node_labels,columns=node_labels)
wdf

,A,B,C,D,I,O
A,0.806,0.000,0.032,0.161,0.000,0.00
B,0.161,0.032,0.000,0.806,0.005,0.00
C,0.032,0.806,0.161,0.000,0.000,0.00
D,0.000,0.161,0.806,0.032,0.000,0.05
I,0.000,0.050,0.000,0.000,0.000,0.00
O,0.000,0.000,0.000,0.005,0.000,0.00


In [6]:
sbs = np.round(pyphi.convert.state_by_node2state_by_state(network.tpm),3)
states = [utils.strp(str(s)) for s in list(pyphi.utils.all_states(network.size))]

import plotly.express as px

fig = px.imshow(sbs,x=states,y=states,color_continuous_scale='Greys')
fig.update_xaxes(side="top")
fig.update_layout(height=1200,width=2000,xaxis={'visible': False, 'showticklabels': False},yaxis={'visible': False, 'showticklabels': False})
# fig.show()
fig.write_image(f'figures/ch3/{net_name}_tpm_nolabels.png')



fig = px.imshow(sbs,x=states,y=states,color_continuous_scale='Greys')
fig.update_xaxes(side="top")
fig.update_layout(height=1200,width=2000)
# fig.show()
fig.write_image(f'figures/ch3/{net_name}_tpm_labels.png')

# standard system

In [7]:

state = (0, 1, 0, 0, 1, 0)
subsystem_standard = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_standard, parallel=True)]
)

standard_ces = compute.filter_ces(subsystem_standard, ces, None, max_relations_k=3, parallel=True)


  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend LokyBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    2.5s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    3.9s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    4.3s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    4.7s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    5.3s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    5.9s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    6.5s
[Parallel(n_jobs=120)]: Done 181 tasks      | elapsed:    7.0s
[Parallel(n_jobs=120)]: Done 237 out of 432 | elapsed:    8.4s remaining:    6.9s
[Parallel(n_jobs=120)]: Done 281 out of 432 | elapsed:    9.5s remaining:    5.1s
[Parallel(n_jobs=120)]: Done 325 out of 432 | elapsed:   10.1s remaining:    3.3s
[Parallel(n_jobs=120)]: Done 369 out of 432 | elapsed:   10.2s remaining:    1.7s
[Parallel(n_jobs=120)]: Done 413 out of 432 | elapsed:   10.6s remaining:    0.5s
[Parall

In [15]:
# allrels = []
# for k in tqdm(range(2,len(ces)+1)):
#     print(f'Computing k = {k}...')
#     krels = list(pyphi.relations.relations(subsystem_standard,ces,min_order=k,max_order=k,parallel=True,parallel_kwargs=dict(n_jobs=120,batch_size=10000,verbose=5)))
#     allrels.append(krels)

# utils.pklthis(allrels,'ch3_allrels.pkl')

In [16]:
allrels = utils.loadpkl('ch3_allrels.pkl')

allrelsflat = utils.flatten(allrels)

In [19]:
standard_ces_allrels = compute.filter_ces(subsystem_standard, ces, None, relations=allrelsflat, parallel=True, n_jobs=200)

  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=200)]: Using backend LokyBackend with 200 concurrent workers.
/home/mgrasso/miniconda3/envs/4.0/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=200)]: Done  21 tasks      | elapsed: 20.6min
[Parallel(n_jobs=200)]: Done  77 out of 432 | elapsed: 63.0min remaining: 290.3min
[Parallel(n_jobs=200)]: Done 121 out of 432 | elapsed: 95.1min remaining: 244.5min
[Parallel(n_jobs=200)]: Done 165 out of 432 | elapsed: 128.4min remaining: 207.7min
[Parallel(n_jobs=200)]: Done 209 out of 432 | elapsed: 161.4min remaining: 172.2min
[Parallel(n_jobs=200)]: Done 253 out of 432 | elapsed: 188.0min remaining: 133.0min
[Parallel(n_jobs=200)]: Done 297 out of 432 | elapsed: 221.6min remaining: 100.7min
[Parallel(n_jobs=200)]: Done 341 out of 432 | elapsed: 255.4min remaining: 68.2min
[Parallel(n_jobs=2

In [ ]:
pklthis(standard_ces_allrels,f'{net_name}_ces_allrels.pkl')

# Plotting

In [20]:
# # plotting FULL CES
# fig = viz.plot_ces(
#     subsystem_standard,
#     standard_ces[0]["ces"],
#     standard_ces[0]["relations"],
#     network_name="figures/ch3/standard_all",
# )

In [31]:
# plotting MESH
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_mesh",
    png_resolution=(3000,2000),    
    show_legend=False,    
    show_edges=False,
    show_mesh=True,
    show_links=False,
    show_labels=False,
    show_mechanism_labels=False,
    show_purview_labels=False,
    show_mechanism_base=False,
    show_chains=False,    
)

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

In [32]:
# plotting EDGES
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_EDGES",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    show_mesh=False,
    show_links=False,
    show_labels=False,
    show_mechanism_base=False,
    show_chains=False,    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

In [33]:
# plotting LABELS
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_LABELS",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=False,
    show_mesh=False,
    show_links=False,
    show_labels=True,
    show_mechanism_base=False,
    show_chains=False,    
)

In [36]:
# plotting BASE
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_BASE",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    transparent_edges=True,
    show_mesh=False,
    show_links=False,
    show_labels=False,
    show_mechanism_base=True,
    show_chains=True,
    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

In [37]:
# plotting LINKS
fig = viz.plot_ces(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    network_name="figures/ch3/standard_LINKS",
    show_legend=False,    
    png_resolution=(3000,2000),    
    show_edges=True,
    transparent_edges=True,
    show_mesh=False,
    show_links=True,
    show_labels=False,
    show_mechanism_base=False,
    show_chains=False,
    
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

# Information

## Consequence

In [38]:
state = (0, 0, 0, 1, 0, 1)
subsystem_consequence = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_consequence, parallel=True)]
)

consequence_ces = compute.filter_ces(
    subsystem_consequence, ces, None, max_relations_k=3, parallel=True
)

  0%|          | 0/288 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    1.3s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    2.4s
[Parallel(n_jobs=120)]: Done  78 out of 288 | elapsed:    3.5s remaining:    9.5s
[Parallel(n_jobs=120)]: Done 107 out of 288 | elapsed:    4.3s remaining:    7.3s
[Parallel(n_jobs=120)]: Done 136 out of 288 | elapsed:    5.6s remaining:    6.3s
[Parallel(n_jobs=120)]: Done 165 out of 288 | elapsed:    6.3s remaining:    4.7s
[Parallel(n_jobs=120)]: Done 194 out of 288 | elapsed:    7.4s remaining:    3.6s
[Parallel(n_jobs=120)]: Done 223 out of 288 | elapsed:    7.9s remaining:    2.3s
[Parallel(n_jobs=120)]: Done 252 out of 288 | elapsed:    8.4s remaining:    1.2s
[Parallel(n_jobs=120)]: Done 281 out of 288 | elapsed:    9.7s remaining:    0.2s
[Parallel(n_jobs=120)]: Done 288 out of 288 | elapsed:    9.9s fi

In [39]:
# plotting
fig = viz.plot_ces(
    subsystem_consequence,
    consequence_ces[0]["ces"],
    consequence_ces[0]["relations"],
    network_name="figures/ch3/information_1",
    show_legend=False,    
    png_resolution=(3000,2000),       
)

Computing edges:   0%|          | 0/76 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/158 [00:00<?, ?it/s]

## All off

In [40]:
state = (0, 0, 0, 0, 0, 0)
subsystem_all_off = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_all_off, parallel=True)]
)

all_off_ces = compute.filter_ces(
    subsystem_all_off, ces, None, max_relations_k=3, parallel=True
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [41]:
# plotting
fig = viz.plot_ces(
    subsystem_all_off,
    all_off_ces[0]["ces"],
    all_off_ces[0]["relations"],
    network_name="figures/ch3/information_2",
    show_legend=False,    
    png_resolution=(3000,2000),           
)

Computing edges:   0%|          | 0/89 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/180 [00:00<?, ?it/s]

# Integration

In [43]:
plots.plot_effect_of_MIP(
    subsystem_standard,
    standard_ces[0]['ces'],
    standard_ces[0]['relations'],
    "figures/ch3/integration",
    partitions=None,
    common_kwargs=dict(show_legend=False,    
    png_resolution=(3000,2000))
)

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

Not redrawing base


Computing edges:   0%|          | 0/22 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/53 [00:00<?, ?it/s]

Not redrawing base


# Exclusion

## Smaller system

In [85]:

state = (0, 1, 0, 0, 1, 0)
smaller_subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(smaller_subsystem, parallel=True)]
)

smaller_ces = compute.filter_ces(smaller_subsystem, ces, None, max_relations_k=3, parallel=True)


  0%|          | 0/36 [00:00<?, ?it/s]

In [101]:
# plotting
fig = viz.plot_ces(
    smaller_subsystem,
    smaller_ces[0]["ces"],
    smaller_ces[0]["relations"],
    network_name="figures/ch3/exclusion_1",
    show_legend=False,    
    png_resolution=(3000,2000),
    floor_width_scales=[2, 2, 2],    
    floor_height_scale=[1,1,1],
    base_height_scale=1,
    base_z_offset=0.3,
    base_width_scale=1,
    cause_effect_distance=0.3,
    eye_coordinates=(-0.3, -0.2, 0.025),
    
)

Computing edges:   0%|          | 0/6 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/2 [00:00<?, ?it/s]

## Larger system

In [57]:

state = (0, 1, 0, 0, 1, 0)
larger_subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3, 5))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(larger_subsystem, parallel=True)]
)

larger_ces = compute.filter_ces(larger_subsystem, ces, None, max_relations_k=3, parallel=True)


  0%|          | 0/288 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.2s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.7s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    1.0s
[Parallel(n_jobs=120)]: Done  78 out of 288 | elapsed:    1.8s remaining:    4.8s
[Parallel(n_jobs=120)]: Done 107 out of 288 | elapsed:    2.3s remaining:    3.9s
[Parallel(n_jobs=120)]: Done 136 out of 288 | elapsed:    3.1s remaining:    3.4s
[Parallel(n_jobs=120)]: Done 165 out of 288 | elapsed:    3.7s remaining:    2.7s
[Parallel(n_jobs=120)]: Done 194 out of 288 | elapsed:    4.3s remaining:    2.1s
[Parallel(n_jobs=120)]: Done 223 out of 288 | elapsed:    4.6s remaining:    1.4s
[Parallel(n_jobs=120)]: Done 252 out of 288 | elapsed:    5.0s remaining:    0.7s
[Parallel(n_jobs=120)]: Done 281 out of 288 | elapsed:    5.7s remaining:    0.1s
[Parallel(n_jobs=120)]: Done 288 out of 288 | elapsed:    5.8s fi

In [84]:
# plotting
fig = viz.plot_ces(
    larger_subsystem,
    larger_ces[0]["ces"],
    larger_ces[0]["relations"],
    network_name="figures/ch3/exclusion_2",
    show_legend=False,    
    png_resolution=(3000,2000),
    floor_width_scales=[1.25 for x in range(len(list(pyphi.utils.powerset(larger_subsystem.node_indices))))],
    floor_height_scale=[1.75 for x in range(len(list(pyphi.utils.powerset(larger_subsystem.node_indices))))],
    base_height_scale=5,
    base_z_offset=-.2,
     base_width_scale=.6,
#     cause_effect_distance=0.6,    
    hovermode=False,    
    eye_coordinates=(0.2, -0.4, 0.025),
)

Computing edges:   0%|          | 0/37 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/37 [00:00<?, ?it/s]

# For chapter 4

## A compound distinction
The context of a single cause-effect distinction 

In [76]:
fig=plots.compound_distinction(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    (0,),
    "figures/ch3/compound_distinction",
)

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

Not redrawing base


Computing edges:   0%|          | 0/15 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/49 [00:00<?, ?it/s]

Not redrawing base


## A compound relation
The set of relations binding two compound distinctions.
The relations are those where every relatum is in exactly one of the compound distinctions (contexts of purviews specified by a single mechanism).

In [75]:
fig = plots.compound_relation(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    [(0,), (2,)],
    "figures/ch3/compound_relation",
)

[Relation(relata=(Effect[0,1,2,3],Effect[3]), purview=frozenset({2}), phi=0.35450678342719005), Relation(relata=(Cause[2,3],Cause[0,1]), purview=frozenset({1}), phi=0.012551818426460727), Relation(relata=(Cause[0,1,2],Cause[0,1]), purview=frozenset({1, 2}), phi=0.4057259681108796), Relation(relata=(Effect[0,2],Effect[0,3]), purview=frozenset({2}), phi=0.0878285603007224), Relation(relata=(Effect[2,3],Effect[0,2]), purview=frozenset({2}), phi=0.0878285603007224), Relation(relata=(Effect[0,1,2],Effect[0,3]), purview=frozenset({2}), phi=0.052735286416853586), Relation(relata=(Effect[2,3],Effect[0,1,2]), purview=frozenset({2}), phi=0.052735286416853586), Relation(relata=(Effect[0,2],Effect[0,3],Effect[3]), purview=frozenset({2}), phi=0.08782856030072272), Relation(relata=(Effect[2,3],Effect[0,2],Effect[3]), purview=frozenset({2}), phi=0.08782856030072272), Relation(relata=(Effect[0,1,2],Effect[0,3],Effect[3]), purview=frozenset({2}), phi=0.052735286416853905), Relation(relata=(Effect[2,3],

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

Not redrawing base
Not redrawing base


Computing edges:   0%|          | 0/7 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/4 [00:00<?, ?it/s]

# A mechanism phifold

In [7]:
fig = plots.mechanism_phi_fold(
    subsystem_standard,
    standard_ces[0]["ces"],
    standard_ces[0]["relations"],
    [(0, 2,)],
    "figures/ch3/mechanism_phifold",
    )

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

Not redrawing base


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

# A purview phifold

# A compound phifold

# A content

In [ ]:
content(
    system,
    ces,
    relations,
    mices,
    figure_name,
    common_kwargs=dict(),
    uncommon_kwargs=[dict(), dict(), dict()],
):

    all_context_relations = [r for r in relations if any([relatum in mices for relatum in r.relata])] for mices in all_mices]
    all_context_distinctions = [list(set([mice for relation in context_relations for mice in relation.relata])) for context_relations in all_context_relations]

    # Getting the relations that have relata in exactly one of the contexts
    context_biding_relations = [r for r in relations 
                                if all([any([relatum in context_distinctions for relatum in r.relata]) for context_distinctions in all_context_distinctions]) 
                                and all([sum([relatum in context_distinctions for relatum in r.relata])==1 for context_distinctions in all_context_distinctions])]
    required_mices = list(set([mice for relation in context_biding_relations for mice in relation.relata]))
    
    cess = [ces, [mice for mices in all_mices for mice in mices], required_mices]
    relations = [relations, [], context_biding_relations]

    default_common_kwargs = dict(
        network_name=figure_name, show_legend=False, show_labels=False,
    )
    default_common_kwargs.update(common_kwargs)

    default_uncommon_kwargs = [
        dict(
            surface_colorscale="Blues",
            surface_opacity=0.1,
            show_links=False,
            show_edges=False,
            save_plot_to_png=False,
            save_plot_to_html=False,
        ),
        dict(
            surface_colorscale="Greys",
            surface_opacity=0.01,
            show_edges=False,
            show_chains=False,
            show_mechanism_base=False,
            show_mechanism_labels=True,
            show_links=True,
        ),
        dict(
            surface_colorscale="Purples",
            surface_opacity=0.9,
            show_chains=False,
            show_mechanism_base=False,
            matteo_edge_color=False,
            show_links=False,
            show_edges=True,
            show_purview_labels=True,
        ),
    ]
    default_uncommon_kwargs = [
        dict(**default_uncommon_kwarg, **uncommon_kwarg)
        for default_uncommon_kwarg, uncommon_kwarg in zip(
            default_uncommon_kwargs, uncommon_kwargs
        )
    ]

    nonstandard_kwargs = [
        dict(**default_common_kwargs, **kwargs) for kwargs in default_uncommon_kwargs
    ]


# Neurologically relevant manipulations

## Activating/deactivating a unit

In [15]:
state = (0, 1, 0, 0, 1, 0)
subsystem_state1 = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_state1, parallel=True)]
)

ces_state1 = compute.filter_ces(
    subsystem_state1, ces, None, max_relations_k=3, parallel=True
)


Computing concepts:  67%|██████████████████████████████████████████                     | 10/15 [00:04<00:01,  2.88it/s]
                                                                                                                        

  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.7s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    1.1s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    1.4s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    1.8s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    2.2s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    2.7s
[Parallel(n_jobs=120)]: Done 181 tasks      | elapsed:    3.3s
[Parallel(n_jobs=120)]: Done 237 out of 432 | elapsed:    4.3s remaining:    3.5s
[Parallel(n_jobs=120)]: Done 281 out of 432 | elapsed:    5.1s remaining:    2.7s
[Parallel(n_jobs=120)]: Done 325 out of 432 | elapsed:    8.1s remaining:    2.7s
[Parallel(n_jobs=120)]: Done 369 out of 432 | elapsed:    8.6s remaining:    1.5s
[Parallel(n_jobs=120)]: Done 413 out of 432 | elapsed:    9.2s remaining:    0

In [19]:
state = (1, 1, 0, 0, 1, 0)
subsystem_state2 = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_state2, parallel=True)]
)

ces_state2 = compute.filter_ces(
    subsystem_state2, ces, None, max_relations_k=3, parallel=True
)


Computing concepts:  80%|██████████████████████████████████████████████████▍            | 12/15 [00:05<00:00,  3.23it/s]
                                                                                                                        

  0%|          | 0/432 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.3s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.8s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    1.1s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    1.5s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    2.0s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    2.5s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    2.9s
[Parallel(n_jobs=120)]: Done 181 tasks      | elapsed:    3.3s
[Parallel(n_jobs=120)]: Done 237 out of 432 | elapsed:    4.2s remaining:    3.5s
[Parallel(n_jobs=120)]: Done 281 out of 432 | elapsed:    7.5s remaining:    4.0s
[Parallel(n_jobs=120)]: Done 325 out of 432 | elapsed:    8.3s remaining:    2.7s
[Parallel(n_jobs=120)]: Done 369 out of 432 | elapsed:    8.8s remaining:    1.5s
[Parallel(n_jobs=120)]: Done 413 out of 432 | elapsed:    9.3s remaining:    0

In [20]:
# plotting
fig = viz.plot_ces(
    subsystem_state2,
    ces_state2[0]["ces"],
    ces_state2[0]["relations"],
    network_name="figures/ch4/avtivation2",
)

Computing edges:   0%|          | 0/62 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/113 [00:00<?, ?it/s]

In [18]:
# plotting
fig = viz.plot_ces(
    subsystem_state1,
    ces_state1[0]["ces"],
    ces_state1[0]["relations"],
    network_name="figures/ch4/avtivation1",
)

Computing edges:   0%|          | 0/73 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/136 [00:00<?, ?it/s]

## Inactivating a unit (lesion)

## Strengthening/weakening a connection (plasticity)

In [ ]:
new_weights = weights.copy()
new_weights[0,2] = new_weights[0,3]

network = ng.get_net(
    mech_func,
    new_weights,
    exp=exp,
    th=th,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=False,
)

state = (0, 1, 0, 0, 1, 0)
subsystem_new_weight = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3))

ces = utils.flatten(
    [[d.cause, d.effect] for d in pyphi.compute.ces(subsystem_new_weight, parallel=True)]
)

ces_new_weight = compute.filter_ces(
    subsystem_new_weight, ces, None, max_relations_k=3, parallel=True
)

# plotting
fig = viz.plot_ces(
    subsystem_new_weight,
    ces_new_weight[0]["ces"],
    ces_new_weight[0]["relations"],
    network_name="figures/ch4/new_weight",
)

  0%|          | 0/64 [00:00<?, ?it/s]


Computing concepts: 100%|███████████████████████████████████████████████████████████████| 15/15 [00:18<00:00,  3.21it/s]

## Disconnecting a unit

In [23]:
weights[0,2]

0.03225806451612903

## Changing excitability (attention)

In [11]:
!conda install -y networkx

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/bjuel/miniconda3/envs/visualize

  added / updated specs:
    - networkx


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.10.26 |       h06a4308_2         115 KB
    networkx-2.6.3             |     pyhd3eb1b0_0         1.3 MB
    ------------------------------------------------------------
                                           Total:         1.4 MB

The following NEW packages will be INSTALLED:

  networkx           pkgs/main/noarch::networkx-2.6.3-pyhd3eb1b0_0

The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2021.10.~ --> pkgs/main::ca-certificates-2021.10.26-h0

In [ ]:
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph

In [9]:
def save_digraph(
    subsystem,
    element_positions,
    digraph_filename="digraph.png",
    plot_digraph=False,
    layout="dot",
):
    # network = subsystem.network
    G = nx.DiGraph()

    for n in range(subsystem.size):
        node_info = format_node(n, subsystem)
        G.add_node(
            node_info["label"],
            style=node_info["style"],
            fillcolor=node_info["fillcolor"],
            fontcolor=node_info["fontcolor"],
            pos=element_positions[n, :],
        )

    edges = [
        [format_node(i, subsystem)["label"] for i in n]
        for n in np.argwhere(subsystem.cm)
    ]

    G.add_edges_from(edges)
    G.graph["node"] = {"shape": "circle"}
    pos = {
        subsystem.node_labels[n]: tuple(element_positions[n, :])
        for n in range(subsystem.size)
    }
    nx.draw(G, pos)
    # A = to_agraph(G)
    # A.layout(layout)
    # A.draw(digraph_filename)
    if plot_digraph:
        return Image(digraph_filename)

ModuleNotFoundError: No module named 'networkx'

In [220]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c[0]["big phi"],
)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Batch computation too fast (0.1301s.) Setting batch_size=2.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=120)]: Done  43 out of 256 | elapsed:    0.5s remaining:    2.4s
[Parallel(n_jobs=120)]: Done  69 out of 256 | elapsed:    0.6s remaining:    1.7s
[Parallel(n_jobs=120)]: Done  95 out of 256 | elapsed:    0.8s remaining:    1.3s
[Parallel(n_jobs=120)]: Done 121 out of 256 | elapsed:    0.9s remaining:    1.0s
[Parallel(n_jobs=120)]: Done 147 out of 256 | elapsed:    1.1s remaining:    0.8s
[Parallel(n_jobs=120)]: Done 173 out of 256 | elapsed:    1.5s remaining:    0.7s
[Parallel(n_jobs=120)]: Done 199 out of 256 | elapsed:    1.6s remaining:    0.5s
[Parallel(n_jobs=120)]: Done 225 out of 256 | elapsed:    1.7s remaining:    0.2s
[Parallel(n_jobs=120)]: Done 251 out of 256 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=120)]: D

  0%|          | 0/512 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Batch computation too fast (0.0731s.) Setting batch_size=2.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    0.4s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    0.5s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    0.6s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    0.8s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    1.0s
[Parallel(n_jobs=120)]: Done 169 out of 512 | elapsed:    1.1s remaining:    2.3s
[Parallel(n_jobs=120)]: Done 221 out of 512 | elapsed:    1.4s remaining:    1.8s
[Parallel(n_jobs=120)]: Done 273 out of 512 | elapsed:    1.8s remaining:    1.6s
[Parallel(n_jobs=120)]: Done 512 out of 512 | elapsed:    2.4s finished


...

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Batch computation too fast (0.0847s.) Setting batch_size=2.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    0.5s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    0.6s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    0.8s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    1.0s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    1.2s
[Parallel(n_jobs=120)]: Done 169 out of 512 | elapsed:    1.6s remaining:    3.2s
[Parallel(n_jobs=120)]: Done 221 out of 512 | elapsed:    1.8s remaining:    2.3s
[Parallel(n_jobs=120)]: Done 273 out of 512 | elapsed:    2.0s remaining:    1.7s
[Parallel(n_jobs=120)]: Done 429 out of 512 | elapsed:    2.5s remaining:    0.5s
[Parallel(n_jobs=120)]: Done 481 out of 512 | elapsed:   

  0%|          | 0/256 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Batch computation too fast (0.1137s.) Setting batch_size=2.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=120)]: Done  43 out of 256 | elapsed:    0.5s remaining:    2.2s
[Parallel(n_jobs=120)]: Done  69 out of 256 | elapsed:    0.7s remaining:    1.9s
[Parallel(n_jobs=120)]: Done  95 out of 256 | elapsed:    0.9s remaining:    1.5s
[Parallel(n_jobs=120)]: Done 121 out of 256 | elapsed:    1.1s remaining:    1.2s
[Parallel(n_jobs=120)]: Done 147 out of 256 | elapsed:    1.3s remaining:    0.9s
[Parallel(n_jobs=120)]: Done 173 out of 256 | elapsed:    1.6s remaining:    0.8s
[Parallel(n_jobs=120)]: Done 199 out of 256 | elapsed:    1.7s remaining:    0.5s
[Parallel(n_jobs=120)]: Done 225 out of 256 | elapsed:    1.9s remaining:    0.3s
[Parallel(n_jobs=120)]: Done 251 out of 256 | elapsed:    2.0s remaining:    0.0s
[Parallel(n_jobs=120)]: D

  0%|          | 0/512 [00:00<?, ?it/s]

[Parallel(n_jobs=120)]: Using backend MultiprocessingBackend with 120 concurrent workers.
[Parallel(n_jobs=120)]: Batch computation too fast (0.0825s.) Setting batch_size=2.
[Parallel(n_jobs=120)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=120)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=120)]: Done  48 tasks      | elapsed:    0.4s
[Parallel(n_jobs=120)]: Done  73 tasks      | elapsed:    0.6s
[Parallel(n_jobs=120)]: Done  98 tasks      | elapsed:    0.7s
[Parallel(n_jobs=120)]: Done 125 tasks      | elapsed:    0.9s
[Parallel(n_jobs=120)]: Done 152 tasks      | elapsed:    1.1s
[Parallel(n_jobs=120)]: Done 169 out of 512 | elapsed:    1.2s remaining:    2.4s
[Parallel(n_jobs=120)]: Done 221 out of 512 | elapsed:    1.5s remaining:    1.9s
[Parallel(n_jobs=120)]: Done 481 out of 512 | elapsed:    2.8s remaining:    0.2s
[Parallel(n_jobs=120)]: Done 512 out of 512 | elapsed:    3.0s finished


In [221]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/random',surface_opacity=0.7,)

TypeError: list indices must be integers or slices, not str

In [222]:
max_ces[0]['big phi']

0.0

In [223]:
max_ces[0]['MIP']

(((1,), (0, 2, 3)), (0, 2, 3), (1,), EFFECT)

In [224]:
utils.sepces2df(max_ces[0]['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.791118
1,A,EFFECT,AD,00,0.895152
2,B,CAUSE,B,1,0.518791
3,B,EFFECT,B,1,0.617717
4,C,CAUSE,C,0,0.791118
5,C,EFFECT,CD,00,0.895152
6,D,CAUSE,AB,11,0.774729
7,D,EFFECT,D,1,0.031598
8,BC,CAUSE,BC,10,0.100293
9,BC,EFFECT,C,1,0.177346


In [119]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,1,0.756225
1,A,EFFECT,A,1,0.770418
2,B,CAUSE,B,1,0.518791
3,B,EFFECT,B,1,0.617717
4,C,CAUSE,C,1,0.756225
5,C,EFFECT,C,1,0.770418
6,D,CAUSE,ABC,111,0.923806
7,D,EFFECT,D,1,0.050664
8,AB,CAUSE,AB,11,0.100404
9,AB,EFFECT,ABD,111,0.461854


In [135]:
# SAVED RANDOM WEIGHTS
#random_weifgts = weights.copy()
random_weifgts

array([[0.05282703, 0.03575078, 0.08661428, 0.06543685, 0.06009787,
        0.00043135],
       [0.08845203, 0.0769501 , 0.04547727, 0.00414203, 0.09540542,
        0.02368191],
       [0.06478836, 0.04271017, 0.00028466, 0.04821126, 0.02971616,
        0.07927113],
       [0.01130869, 0.0821748 , 0.00165421, 0.09935794, 0.0197395 ,
        0.09792563],
       [0.07471203, 0.07473525, 0.06208848, 0.06050363, 0.035507  ,
        0.08971665],
       [0.04228357, 0.02933043, 0.09749088, 0.05394963, 0.01207341,
        0.02973016]])

In [72]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.486034
1,A,EFFECT,A,0,0.627340
2,B,CAUSE,B,0,0.601248
3,B,EFFECT,B,0,0.703992
4,C,CAUSE,C,0,0.486034
5,C,EFFECT,C,0,0.627340
6,D,CAUSE,BD,00,0.311921
7,D,EFFECT,D,0,0.177053
8,AB,CAUSE,AB,00,0.075913
9,AB,EFFECT,ABD,000,0.185003


In [76]:
CES = [m for f in filtered for m in f if m['big phi']>8.8][0]
utils.sepces2df(CES['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.512357
1,A,EFFECT,A,0,0.649867
2,B,CAUSE,B,1,0.452130
3,B,EFFECT,B,1,0.567426
4,C,CAUSE,C,0,0.514622
5,C,EFFECT,C,0,0.652147
6,AB,CAUSE,AB,10,0.180604
7,AB,EFFECT,ABD,010,0.038629
8,BC,CAUSE,BC,01,0.175186
9,BC,EFFECT,BCD,100,0.038340


In [77]:
CES['MIP']

(((0,), (1, 2, 3)), (1, 2, 3), (0,), EFFECT)

In [69]:
good_weights = weights
good_weights

array([[0.80429078, 0.25524274, 0.00307273, 0.3515403 , 0.00492981,
        0.00996336],
       [0.25475904, 0.8015509 , 0.25453847, 0.40018269, 0.01702802,
        0.0043879 ],
       [0.00997719, 0.25221898, 0.80581981, 0.35990666, 0.00266345,
        0.00728655],
       [0.01578615, 0.01329649, 0.0181051 , 0.60958749, 0.00620138,
        0.35398605],
       [0.00974372, 0.35204232, 0.00702253, 0.00453444, 0.00349476,
        0.00514265],
       [0.00515462, 0.00230007, 0.00979452, 0.00469913, 0.00642495,
        0.00897918]])

In [163]:
# conjunction disjunction
# ch3 system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 45
x0 = 0.7

noise = 0.00

s = 0.8
l = 0.25
f = 0.0
q = 0.075
j = 1 - q
i = j - 0.05
p = 0.01
k = (1 - p)/2
b = 0.001

weights = np.array(
    [
        [s, l, k, i, 0, 0],  # A
        [l, s, k, j, 0, 0],  # B
        [0, 0, p, 0, 0, i],  # C
        [0, 0, 0, q, 0, j],  # D
        [0, i, 0, 0, 0, 0],  # I
        [i, 0, 0, 0, 0, q],  # O
        # A, B, C, D, I, O
    ]
)

rule = (weights>0)
weights += rule * np.random.random((6,6))*noise
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = [f for f in "sslglg"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 1, 1, 1)
#state = (0,)*6
subsystem = pyphi.Subsystem(network, state, nodes=(0,1,2,3))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [164]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))


Computing concepts: 100%|███████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  8.43it/s]
                                                                                                                        

there are 4 possible compositional states


In [165]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

  0%|          | 0/1440 [00:00<?, ?it/s]

  0%|          | 0/480 [00:00<?, ?it/s]

In [145]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/con_disj',surface_opacity=0.7,)

Computing edges:   0%|          | 0/15 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/8 [00:00<?, ?it/s]

In [166]:
max_ces['big phi']

0.0

In [167]:
max_ces['MIP']

(((0,), (1, 2, 3)), (1, 2, 3), (0,), CAUSE)

In [168]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,AB,00,0.999782
1,A,EFFECT,C,0,0.415041
2,B,CAUSE,B,1,0.091296
3,B,EFFECT,ABC,111,0.575169


In [169]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,AB,00,0.999782
1,A,EFFECT,C,0,0.415041
2,B,CAUSE,B,1,0.091296
3,B,EFFECT,ABC,111,0.575169
4,C,CAUSE,A,0,0.276717
5,C,EFFECT,C,0,0.000020
6,D,CAUSE,AB,01,0.503393
7,D,EFFECT,D,1,0.016511
8,AB,CAUSE,AB,01,0.117290
9,AB,EFFECT,D,1,0.954088


In [ ]:
# ch3 larger system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

noise = 0.02
rule = (weights>0)

s = 0.8
l = 0.25
f = 0.0
q = 0.6
j = 1 - q
i = j - 0.05
b = 0.01

weights = np.array(
    [
        [s, l, f, 0, i, 0],  # A
        [l, s, l, f, j, b],  # B
        [f, l, s, l, i, 0],  # C
        [0, f, l, s, i, 0],  # D
        [b, b, b, b, q, 0],  # E
        [0, i, 0, 0, 0, 0],  # I
       # A, B, C, D, E, I
    ]
)

weights += rule * np.random.random((6,6))*noise
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "E", "I"]
mech_func = [f for f in "ssssgl"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 1, 0, 0, 1, 1)
subsystem = pyphi.Subsystem(network, state, nodes=(0, 1, 2, 3, 4))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [130]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))



Computing concepts:   0%|                                                                        | 0/31 [00:00<?, ?it/s]

Computing concepts:   3%|██                                                              | 1/31 [00:02<01:22,  2.73s/it]

Computing concepts:  26%|████████████████▌                                               | 8/31 [00:02<00:06,  3.61it/s]

Computing concepts:  52%|████████████████████████████████▌                              | 16/31 [00:03<00:01,  7.59it/s]

Computing concepts:  77%|████████████████████████████████████████████████▊              | 24/31 [00:03<00:00, 12.92it/s]

Computing concepts:  94%|██████████████████████████████████████████████████████████▉    | 29/31 [00:04<00:00,  9.61it/s]

                                                                                                                        

there are 384 possible compositional states


In [131]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/384 [00:00<?, ?it/s]

  0%|          | 0/192 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [120]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/max_ces',surface_opacity=0.7,)

Computing edges:   0%|          | 0/33 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/32 [00:00<?, ?it/s]

In [113]:
mice = max_ces['ces'][0]

In [118]:
mice.ria.node_labels

NodeLabels(('A', 'B', 'C', 'D', 'I', 'O'))

In [109]:
max_ces['big phi']

4.375449186656582e+00

In [110]:
max_ces['MIP']

(((2,), (0, 1, 3)), (0, 1, 3), (2,), EFFECT)

In [111]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.502959
1,A,EFFECT,A,0,0.641988
2,B,CAUSE,B,1,0.460940
3,B,EFFECT,B,1,0.574265
4,C,CAUSE,C,0,0.497810
5,C,EFFECT,C,0,0.637561
6,AB,CAUSE,AB,10,0.181306
7,AB,EFFECT,ABD,010,0.039542
8,BC,CAUSE,BC,01,0.181514
9,BC,EFFECT,BCD,100,0.039336


In [199]:
# ch3 system creation
net_name = "ch3_v5"

th = 1 / 4
exp = 5
mu = 1
si = 0.3
ll = 1
kk = 15
x0 = 0.7

noise = 0.00

s = 0.8
l = 0.4
f = 0.2
q = 0.05
j = (1 - q)/3
i = j - 0.00
b = 0.01

weights = np.array(
    [
        [s, l, f, l, 0, 0],  # A
        [l, s, l, f, b, 0],  # B
        [f, l, s, l, 0, 0],  # C
        [l, f, l, s, 0, i],  # D
        [0, 0, 0, 0, 0, 0],  # I
        [0, 0, 0, 0, 0, 0],  # O
        # A, B, C, D, I, O
    ]
)

rule = (weights>0)
weights += rule * np.random.random((6,6))*noise
weights = np.abs(weights)

node_labels = ["A", "B", "C", "D", "I", "O"]
mech_func = [f for f in "ssssll"]

network = ng.get_net(
    mech_func,
    weights,
    exp=exp,
    th=th,
    mu=mu,
    si=si,
    l=ll,
    k=kk,
    x0=x0,
    node_labels=node_labels,
    network_name=net_name,
    pickle_network=True,
)

state = (0, 0, 0, 0, 1, 1)
#state = (0,)*6
subsystem = pyphi.Subsystem(network, state, nodes=(0,1,2,3))

  0%|          | 0/64 [00:00<?, ?it/s]

Network saved to: ch3_v5


In [200]:
ces = utils.flatten([[d.cause,d.effect] for d in pyphi.compute.ces(subsystem,parallel=True)])
all_compositional_states = compute.get_all_compositional_states(ces)
print('there are {} possible compositional states'.format(len(all_compositional_states)))

there are 2 possible compositional states


In [201]:
max_ces = max(
    [
        compute.filter_ces(subsystem, ces, compositional_state, max_relations_k=3)
        for compositional_state in tqdm(all_compositional_states)
    ],
    key=lambda c: c["big phi"],
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [202]:

fig = viz.plot_ces(subsystem,max_ces['ces'],max_ces['relations'],network_name='figures/toroidal',surface_opacity=0.7,)

Computing edges:   0%|          | 0/60 [00:00<?, ?it/s]

Computing triangles:   0%|          | 0/120 [00:00<?, ?it/s]

In [203]:
max_ces['big phi']

224.29413261759436

In [204]:
max_ces['MIP']

(((0, 1), (2, 3)), (0, 1), (2, 3), CAUSE)

In [205]:
utils.sepces2df(max_ces['ces'],subsystem)

,mechanism,direction,purview,state,phi
0,C,CAUSE,C,0,0.686247
1,C,EFFECT,C,0,0.602082
2,D,CAUSE,D,0,0.686247
3,D,EFFECT,D,0,0.602082
4,AC,CAUSE,B,0,0.187623
5,AC,EFFECT,ABC,000,0.237504
6,AD,CAUSE,AB,00,0.113573
7,AD,EFFECT,AD,00,0.506476
8,BD,CAUSE,A,0,0.187623
9,BD,EFFECT,ABD,000,0.237504


In [198]:
utils.sepces2df(ces,subsystem)

,mechanism,direction,purview,state,phi
0,A,CAUSE,A,0,0.476482
1,A,EFFECT,A,0,0.619804
2,B,CAUSE,B,0,0.476482
3,B,EFFECT,B,0,0.619804
4,C,CAUSE,C,0,0.476482
5,C,EFFECT,C,0,0.619804
6,D,CAUSE,D,0,0.476482
7,D,EFFECT,D,0,0.619804
8,AB,CAUSE,AB,00,0.038655
9,AB,EFFECT,ABCD,0000,0.007057
